In [28]:
import numpy as n

import os
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib as mpl
import seaborn as sns
import pandas as pd
import argparse
import anndata as ad
import scanpy as sc
from scipy.stats import pearsonr

In [30]:
cell_info = pd.read_csv("../data/tosches/metadata.csv", delimiter=' ')
counts = pd.read_csv("../data/tosches/counts.csv", delimiter=' ') # gene x samples
# Select samples for which we have cell info
cell_names = cell_info.index

NameError: name 'np' is not defined

In [32]:
import numpy as np
counts = counts.loc[:, np.array(cell_names)]


In [33]:
adata = ad.AnnData(X = counts.T, dtype = int, obs = cell_info)
# Select ins
adata = adata[[cluster.startswith("i") for cluster in adata.obs['clusters']]]
# Mouse gene names
adata.var_names = [gene[0] + gene[1:].lower() for gene in adata.var_names]

In [34]:
# Map to putative mammalian homologues (Tosches Fig 5)
map = {}
for cluster in range(1, 7):
    map[f'i0{cluster}'] = "Meis2"
for cluster in range(7, 11):
    if cluster<10:
        map[f'i0{cluster}'] = "Sst"
    else:
        map[f'i{cluster}'] = "Sst"
for cluster in range(11, 14):
    map[f'i{cluster}'] = "Pvalb"
for cluster in range(14, 19):
    map[f'i{cluster}'] = "Vip"
# Consistent order
print(adata.obs.columns)

Index(['nGene', 'nUMI', 'orig.ident', 'sampleident', 'areaident',
       'libraryident', 'animalident', 'percent.mito', 'pCODING', 'pUTR',
       'pINTRONIC', 'pINTERGENIC', 'pMITO', 'pallial.area', 'clusters'],
      dtype='object')


In [41]:
adata.obs["Subclass"] = adata.obs["clusters"].map(map)

In [44]:
def organize_subclass_labels(adata):
    """
    Add labels for missing subclasses, 
    order according to mouse tPC1
    """
    subclass_order = ['Pvalb', 'Sst', 'Lamp5', 'Vip', 'Sncg', 'Meis2']
    adata.obs['Subclass'] = adata.obs['Subclass'].astype("category")
    missing_subclasses = [subclass for subclass in subclass_order if subclass not in adata.obs['Subclass'].cat.categories]
    adata.obs['Subclass'] = adata.obs['Subclass'].cat.add_categories(missing_subclasses)
    adata.obs['Subclass'] = adata.obs['Subclass'].cat.reorder_categories(subclass_order)
    return adata

In [45]:
adata = organize_subclass_labels(adata)  

In [2]:
adata = ad.read_h5ad("../data/anndata/bugeon.h5ad")

In [26]:
by_subtype = adata.obs[['Subtype', 'state_modulation']].groupby("Subtype").mean()